# Case Técnico de Data Analysis - iFood

In [2]:
# Instruções para execução
# 1. Instalar o Python 3.8 ou superior
# 2. Instalar o pip (gerenciador de pacotes do Python) 
# 3. Instalar as bibliotecas necessárias com o comando:
##    pip install -r requirements.txt
# 4. Executar o script na ordem

## Importação dos pacotes necessários

In [14]:
import gzip
import json
import awscli
import boto3
import pandas as pd
import pyspark as ps
from pyspark.sql import SparkSession
import requests
import os
import tarfile
import findspark
import spark
from pyspark.sql import SQLContext
import gzip
import json
import pandas as pd
import os
pd.options.display.float_format = '{:.2f}'.format  #Essa linha foi adicionada posteriormente para formatar os números com duas casas decimais e tirar notações científicas



## Importação das bases de dados

In [4]:
# A importação da base de ordes tive que realizar manualmente, pois o arquivo é muito grande e não consegui fazer o download dele diretamente do S3.

In [5]:
# Caminhos
entrada = r"C:\Users\Ítalo\Desktop\Estudos\case_tec_data\dados_ifood\order.json.gz"
saida_dir = r"C:\Users\Ítalo\Desktop\Estudos\case_tec_data\dados_ifood\parquet_chunks"

# Cria pasta de saída se não existir
os.makedirs(saida_dir, exist_ok=True)

batch_size = 100_000  # Tamanho do bloco
dados = []
batch_id = 0

with gzip.open(entrada, 'rt', encoding='utf-8') as f:
    for i, linha in enumerate(f, 1):  # começa do 1 pra debug elegante
        try:
            dados.append(json.loads(linha))
        except Exception as e:
            print(f"Erro na linha {i}: {e}")

        # Quando atingir o tamanho do lote, salva e limpa
        if i % batch_size == 0:
            df_temp = pd.DataFrame(dados)
            # Salva como parquet usando pyarrow
            df_temp.to_parquet(os.path.join(saida_dir, f"order_chunk_{batch_id}.parquet"), engine='pyarrow')
            print(f"Salvo chunk {batch_id} com {len(df_temp)} linhas.")
            dados = []
            batch_id += 1

# Salva o último pedaço, se sobrou algo
if dados:
    df_temp = pd.DataFrame(dados)
    # Salva como parquet usando pyarrow
    df_temp.to_parquet(os.path.join(saida_dir, f"order_chunk_{batch_id}.parquet"), engine='pyarrow')
    print(f"Salvo chunk {batch_id} com {len(df_temp)} linhas.")

Salvo chunk 0 com 100000 linhas.
Salvo chunk 1 com 100000 linhas.
Salvo chunk 2 com 100000 linhas.
Salvo chunk 3 com 100000 linhas.
Salvo chunk 4 com 100000 linhas.
Salvo chunk 5 com 100000 linhas.
Salvo chunk 6 com 100000 linhas.
Salvo chunk 7 com 100000 linhas.
Salvo chunk 8 com 100000 linhas.
Salvo chunk 9 com 100000 linhas.
Salvo chunk 10 com 100000 linhas.
Salvo chunk 11 com 100000 linhas.
Salvo chunk 12 com 100000 linhas.
Salvo chunk 13 com 100000 linhas.
Salvo chunk 14 com 100000 linhas.
Salvo chunk 15 com 100000 linhas.
Salvo chunk 16 com 100000 linhas.
Salvo chunk 17 com 100000 linhas.
Salvo chunk 18 com 100000 linhas.
Salvo chunk 19 com 100000 linhas.
Salvo chunk 20 com 100000 linhas.
Salvo chunk 21 com 100000 linhas.
Salvo chunk 22 com 100000 linhas.
Salvo chunk 23 com 100000 linhas.
Salvo chunk 24 com 100000 linhas.
Salvo chunk 25 com 100000 linhas.
Salvo chunk 26 com 100000 linhas.
Salvo chunk 27 com 100000 linhas.
Salvo chunk 28 com 100000 linhas.
Salvo chunk 29 com 10000

In [6]:
import glob

# Lista todos os arquivos .parquet no diretório de saída
arquivos = glob.glob(saida_dir + r"\order_chunk_*.parquet")

# Concatena os arquivos parquet em um único DataFrame
df_orders = pd.concat([pd.read_parquet(arquivo, engine='pyarrow') for arquivo in arquivos], ignore_index=True)

# Exibe o tamanho e as primeiras linhas
print(df_orders.shape)
df_orders.head()

(3670826, 22)


,cpf,customer_id,customer_name,delivery_address_city,delivery_address_country,delivery_address_district,delivery_address_external_id,delivery_address_latitude,delivery_address_longitude,delivery_address_state,...,merchant_id,merchant_latitude,merchant_longitude,merchant_timezone,order_created_at,order_id,order_scheduled,order_total_amount,origin_platform,order_scheduled_date
0,80532101763,7ba88a68bb2a3504c6bd37a707af57a0b8d6e110a551c7...,GUSTAVO,FRANCA,BR,JARDIM ESPRAIADO,6736655,-47.39,-20.55,SP,...,a992a079a651e699d9149423761df2427c0e3af0a2a1b5...,-47.39,-20.55,America/Sao_Paulo,2019-01-17T22:50:06.000Z,33e0612d62e5eb42aba15b58413137e441fbe906de2feb...,False,46.0,ANDROID,None
1,43352103961,078acecdcf7fa89d356bfa349f14a8219db1ee161ce28a...,MICHELLE,SANTOS,BR,CAMPO GRANDE,8759216,-46.34,-23.96,SP,...,5152f28ee0518b8803ccf0a4096eb2ff8b81e9491861c9...,-46.34,-23.96,America/Sao_Paulo,2019-01-17T17:51:26.000Z,148c4353a2952f3fe7973547283265eb22b575fb712ed2...,False,104.5,ANDROID,None
2,38650991217,0e38a3237b5946e8ab2367b4f1a3ae6e77f1e215bc760c...,VICTOR,GUARULHOS,BR,JARDIM ROSSI,8765930,-46.53,-23.44,SP,...,b6096419455c35d06105a5ef0d25c51f9dd40e1e99ac33...,-46.53,-23.44,America/Sao_Paulo,2019-01-17T22:53:47.000Z,c37e495a91b498bb7b70a9e09ac115d0cdd443f152dc11...,False,35.0,IOS,None
3,63579726866,cab1a004b7206d07910092c515a79834fea0a03d7d9054...,ANNIE,SAO PAULO,BR,PARQUE SAO JORGE,7834087,-46.57,-23.53,SP,...,082bfdcdf6ccdc343e3c4d25ee376b5b6ca7e96ad8b04e...,-46.57,-23.53,America/Sao_Paulo,2019-01-17T23:56:53.000Z,b4df94142d21354611247da9ca94f870c09b93989b531a...,False,40.8,IOS,None
4,90617788806,aa7edf5b166b8c843aec3b96dc561222888734f3879123...,DANIEL,VITORIA,BR,JARDIM CAMBURI,7211683,-40.27,-20.25,ES,...,d7adb764bac29ccb77fb8f746ffbd531bf05ec30a7e130...,-40.27,-20.25,America/Sao_Paulo,2019-01-17T23:40:53.000Z,4ff64b33b272c1886df21b63272220af6a82d1667dba70...,False,48.5,ANDROID,None


In [7]:
# Importando costumers

# Caminho para o arquivo CSV compactado
caminho_consumer = r"C:\Users\Ítalo\Desktop\Estudos\case_tec_data\dados_ifood\consumer.csv.gz"

# Lê o arquivo CSV compactado com pandas
df_consumer = pd.read_csv(caminho_consumer, compression='gzip')

# Exibe as primeiras linhas
print("Consumer DataFrame:")
df_consumer.head()

Consumer DataFrame:


,customer_id,language,created_at,active,customer_name,customer_phone_area,customer_phone_number
0,e8cc60860e09c0bb19610b06ced69c973eb83982cfc98e...,pt-br,2018-04-05T14:49:18.165Z,True,NUNO,46,816135924
1,a2834a38a9876cf74e016524dd2e8c1f010ee12b2b684d...,pt-br,2018-01-14T21:40:02.141Z,True,ADRIELLY,59,231330577
2,41e1051728eba13341136d67d0757f8d8cc44b2a405b71...,pt-br,2018-01-07T03:47:15.554Z,True,PAULA,62,347597883
3,8e7c1dcb64edf95c935147f6d560cb068c44714cb1b21b...,pt-br,2018-01-10T22:17:08.160Z,True,HELTON,13,719366842
4,7823d4cf4150c5daeae0bd799206852fc123bdef0cf5d1...,pt-br,2018-04-06T00:16:20.935Z,True,WENDER,76,543232158


In [8]:
# Importando restaurant

# Caminho para o arquivo CSV compactado
caminho_restaurant = r"C:\Users\Ítalo\Desktop\Estudos\case_tec_data\dados_ifood\restaurant.csv.gz"

# Lê o arquivo CSV compactado com pandas
df_restaurant = pd.read_csv(caminho_restaurant, compression='gzip')

# Exibe as primeiras linhas
print("Restaurant DataFrame:")
df_restaurant.head()

Restaurant DataFrame:


,id,created_at,enabled,price_range,average_ticket,takeout_time,delivery_time,minimum_order_value,merchant_zip_code,merchant_city,merchant_state,merchant_country
0,d19ff6fca6288939bff073ad0a119d25c0365c407e9e5d...,2017-01-23T12:52:30.910Z,False,3,60.0,0,50.0,30.0,14025,RIBEIRAO PRETO,SP,BR
1,631df0985fdbbaf27b9b031a8f381924e3483833385748...,2017-01-20T13:14:48.286Z,True,3,60.0,0,0.0,30.0,50180,SAO PAULO,SP,BR
2,135c5c4ae4c1ec1fdc23e8c649f313e39be8db913d8bc5...,2017-01-23T12:46:33.457Z,True,5,100.0,0,45.0,10.0,23090,RIO DE JANEIRO,RJ,BR
3,d26f84c470451f752bef036c55517b6d9950d41806f10e...,2017-01-20T13:15:04.806Z,True,3,80.0,0,0.0,18.9,40255,SALVADOR,BA,BR
4,97b9884600ea7192314580d9115f8882b8634f5aa201ff...,2017-01-20T13:14:27.701Z,True,3,60.0,0,0.0,25.0,64600,BARUERI,SP,BR


In [9]:
# Importando ab_test_ref

# Caminho para o arquivo CSV compactado
caminho_ab_test = r"C:\Users\Ítalo\Desktop\Estudos\case_tec_data\dados_ifood\ab_test_ref.csv"

# Lê o arquivo CSV compactado com pandas
df_ab_test = pd.read_csv(caminho_ab_test)

# Exibe as primeiras linhas
print("A/B Test DataFrame:")
df_ab_test.head()

A/B Test DataFrame:


,customer_id,is_target
0,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target
1,b821aa8372b8e5b82cdc283742757df8c45eecdd72adf4...,control
2,d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc0...,control
3,6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c...,target
4,dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f...,control


### Criando uma data frame analitico com as informações enriquecidas

In [12]:
df_anl = pd.merge(df_orders, df_consumer, left_on='customer_id', right_on='customer_id', how='inner')

In [18]:
df_anl = df_anl.drop(columns=['customer_name_y'])

In [27]:
df_anl_comp = pd.merge(df_anl, df_ab_test, left_on='customer_id', right_on='customer_id', how='right')

In [28]:
df_anl_comp

,cpf,customer_id,customer_name_x,delivery_address_city,delivery_address_country,delivery_address_district,delivery_address_external_id,delivery_address_latitude,delivery_address_longitude,delivery_address_state,...,order_scheduled,order_total_amount,origin_platform,order_scheduled_date,language,created_at,active,customer_phone_area,customer_phone_number,is_target
0,50859613467,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,10.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target
1,28733920290,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,12.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target
2,19497924963,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,13.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target
3,23451851418,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,13.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target
4,89191850129,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,13.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3661356,47189679416,b204e707910c9d439f8a144c5799dfc0a34aa116c712c6...,Deme,CAMPINAS,BR,CENTRO,8818059,-47.07,-22.90,SP,...,False,49.90,DESKTOP,None,pt-br,2018-04-06T04:00:47.944Z,True,70.00,109839574.00,target
3661357,30748028170,0eeea5f0ef562133be19c93f3c69ae5ef6bac970636605...,DAVID,FORTALEZA,BR,CENTRO,8896198,-38.52,-3.73,CE,...,False,51.80,ANDROID,None,pt-br,2018-01-04T21:55:14.445Z,True,93.00,125556009.00,target
3661358,33702013506,2c58fb2b18f606a19156d4e67d727109660701a112c0af...,TIAGO,LONDRINA,BR,NAO CADASTRADO,8575158,-51.17,-23.31,PR,...,False,47.40,ANDROID,None,pt-br,2018-01-05T22:05:31.168Z,True,30.00,150664610.00,target
3661359,42159730290,5d7242c38e0d2958810ca41c82fd734d2c80d11d03d2f7...,MARCEL,RIO DE JANEIRO,BR,PECHINCHA,8649364,-43.35,-22.92,RJ,...,False,38.00,ANDROID,None,pt-br,2018-04-06T03:48:15.956Z,True,85.00,482169535.00,control


In [29]:
# Dividir o DataFrame entre grupo de teste e controle
grupo_teste = df_anl_comp[df_anl_comp['is_target'] == 'target']
grupo_controle = df_anl_comp[df_anl_comp['is_target'] == 'control']

### Realizando os cálculos necessários para os indicadores

In [30]:
grupo_teste.head()

,cpf,customer_id,customer_name_x,delivery_address_city,delivery_address_country,delivery_address_district,delivery_address_external_id,delivery_address_latitude,delivery_address_longitude,delivery_address_state,...,order_scheduled,order_total_amount,origin_platform,order_scheduled_date,language,created_at,active,customer_phone_area,customer_phone_number,is_target
0,50859613467,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,10.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target
1,28733920290,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,12.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target
2,19497924963,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,13.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target
3,23451851418,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,13.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target
4,89191850129,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,MARCELO,CAMPINAS,BR,CAMBUI,8095398,-47.05,-22.90,SP,...,False,13.00,ANDROID,None,pt-br,2018-04-06T02:48:42.887Z,True,39.00,382036887.00,target


In [31]:
# Número de usuários que realizaram pelo menos um pedido
taxa_conversao_teste = grupo_teste[grupo_teste['order_id'].notnull()]['customer_id'].nunique() / grupo_teste['customer_id'].nunique()
taxa_conversao_controle = grupo_controle[grupo_controle['order_id'].notnull()]['customer_id'].nunique() / grupo_controle['customer_id'].nunique()
print(f"Taxa de conversão do grupo de teste: {taxa_conversao_teste:.2%}")
print(f"Taxa de conversão do grupo de controle: {taxa_conversao_controle:.2%}")

Taxa de conversão do grupo de teste: 99.96%
Taxa de conversão do grupo de controle: 99.96%


In [32]:
# AOV (Valor médio de pedido)
aov_teste = grupo_teste.groupby('customer_id')['order_total_amount'].sum().mean()
aov_controle = grupo_controle.groupby('customer_id')['order_total_amount'].sum().mean()
print(f"AOV do grupo de teste: {aov_teste:.2f}")
print(f"AOV do grupo de controle: {aov_controle:.2f}")

AOV do grupo de teste: 228.68
AOV do grupo de controle: 202.60


In [33]:
# Taxa de Retenção (Usuários com mais de um pedido)
retencao_teste = grupo_teste.groupby('customer_id').size().gt(1).mean()
retencao_controle = grupo_controle.groupby('customer_id').size().gt(1).mean()
print(f"Taxa de retenção do grupo de teste: {retencao_teste:.2%}") 
print(f"Taxa de retenção do grupo de controle: {retencao_controle:.2%}")

Taxa de retenção do grupo de teste: 79.47%
Taxa de retenção do grupo de controle: 74.68%


In [34]:
# Número médio de pedidos por usuário
pedidos_por_usuario_teste = grupo_teste.groupby('customer_id').size().mean()
pedidos_por_usuario_controle = grupo_controle.groupby('customer_id').size().mean()
print(f"Número médio de pedidos por usuário no grupo de teste: {pedidos_por_usuario_teste:.2f}")
print(f"Número médio de pedidos por usuário no grupo de controle: {pedidos_por_usuario_controle:.2f}")

Número médio de pedidos por usuário no grupo de teste: 4.79
Número médio de pedidos por usuário no grupo de controle: 4.23
